# Importing and Splitting Dataset

In [1]:
!pip install emnist

In [2]:
import tensorflow as tf
import numpy as np
from emnist import extract_training_samples, extract_test_samples
#extracting data from emnist dataset
x_train, y_train = extract_training_samples('letters')
x_test, y_test = extract_test_samples('letters')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
print(x_train.shape)# Since we are using emnist dataset no need to make the image dimensions uniform 
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(y_train.dtype) # y_test will also have the same datatype
img_rows = img_cols = 28
input_shape = (img_rows,img_cols,1)
x_train , x_test= x_train.reshape(x_train.shape[0], img_rows, img_cols,1),x_test.reshape(x_test.shape[0], img_rows, img_cols,1)


In [ ]:
#displaying random image 
import matplotlib.pyplot as plt
sample_img=x_train[27]
plt.imshow(sample_img.reshape(28,28),cmap='gray')
plt.show()
print(y_train[27])

In [ ]:
#Normalizing dataset 
x_train=tf.keras.utils.normalize(x_train,axis=1) 
x_test=tf.keras.utils.normalize(x_test,axis=1)
sample_img=x_train[27]
plt.imshow(sample_img.reshape(28,28),cmap='gray')
plt.show() # after normalizing the intensity of the image decreases

# One HOT encoding the labels

In [ ]:
#checking the no. of distinct elements in the labels 
depth_train = np.unique(y_train).size
depth_test = np.unique(y_test).size

print(type(depth_train))
print(type(depth_test))

#making the indices begin from 0
y_train = y_train - 1
y_test = y_test - 1

#printing the labels
print(depth_train)
print(np.unique(y_train))
print(depth_test)
print(np.unique(y_test))
num_classes=depth_train= depth_test


In [ ]:
#converting class vectors to binary class vectors
y_train= tf.one_hot(y_train,depth_train,on_value=1.0, off_value=0.0,axis=-1)
y_test= tf.one_hot(y_test,depth_test,on_value=1.0, off_value=0.0,axis=-1)
print(y_train)

# Creating the model 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time
#Testing various CNN architectures 
conv_layers=[3]
conv_layer_sizes=[64]
dense_layer_sizes=[128]
dense_layers=[3]

for conv_layer in conv_layers:
    for conv_layer_size in conv_layer_sizes:
        for dense_layer in dense_layers:
            for dense_layer_size in dense_layer_sizes:
                NAME=f"{conv_layer}-clayers-{conv_layer_size}-csize-{dense_layer}-dlayers-{dense_layer_size}-dsize-{int(time.time())}"
                tensorboard=TensorBoard(log_dir=f"logs/{NAME}")
                model = Sequential()
                model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))#input layer
                for i in range(conv_layer-1):
                
                    model.add(Conv2D(conv_layer_size, (3, 3), activation='relu'))
                    model.add(MaxPooling2D(pool_size=(2, 2))) 
                    model.add(Dropout(0.5))            
                
                model.add(Flatten())                
                model.add(Dense( dense_layer_size, activation='relu'))
                model.add(Dropout(0.25)) #prevent overfitting
                for i in range(dense_layer-1):

                    model.add(Dense( dense_layer_size, activation='relu'))
                    model.add(Dropout(0.25))
                    
                model.add(Dense(num_classes, activation='softmax'))#final output layer
                # Compile the model
                model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
                history=model.fit(x_train,y_train,batch_size=128,epochs =10,validation_split=0.25,callbacks =[tensorboard],shuffle=True)
                model.save(f"emnist_cnn_model_{NAME}.h5")

# Plotting the graphs of accuracy and loss

In [1]:
print(history.history.keys())# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: name 'history' is not defined